In [ ]:
print("Hello world")

In [16]:
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()


client = OpenAI()
question = input("Enter your query: ")
prompt = f""" You are a data analyst.

There are 3 datasets and user will ask questions based on these datasets:
- sample_bo_tbl_large.csv: country (a 3 digit code like "AAA"), date, total_mins, international_mins, sms, total_data_usage, payg_amount
- sample_sub_details_large.csv: country, channel, date, subs, netadds, churn
- sample_revenue_large.csv: country, channel, date, revenue, net_revenue

Your task:
- Identify relevant datasets (with alias)
- Suggest joins (if needed): left, right, on, how
- Provide filters as list of {{"column": "...", "operator": "...", "value": ...}}
- Indicate aggregation type ("sum", "mean", "correlation", etc.)
- List involved columns

Your response should be only a JSON in the following format which will be an input as parameters to a method which performs these operations using pandas:
{{
  "datasets": ["name":..., "alias":...],
  "joins": [...],
  "filters": [...],
  "aggregation": "...",
  "columns": [...]
}}
If there is a date in the filters convert it into relevant dates of format dd-mm-yyyy and add it to the JSON.
Think step by step and then form the params JSON.
Double check all the values before giving the response. Donot confuse between operators == and =.
If the user asks queries that cannot be answered using the datasets, return a json in the following format:
{{
    "error":"the query cannot be answered using the available datasets"
}}
User Question: {question}"""
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role":"user","content":prompt}],
)
print(response.choices[0].message.content)

```json
{
    "error":"the query cannot be answered using the available datasets"
}
```


In [106]:
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()


client = OpenAI()
question = "What was the churn in AAA last month?"
prompt = f""" You are a specialist in generating queries for SQLite database.

There are 3 tables in the database and user will ask questions based on data from these tables:
- sample_bo_tbl_large: country (a 3 digit code like "AAA"), date, total_mins, international_mins, sms, total_data_usage, payg_amount
- sample_sub_details_large: country, channel, date, subs, netadds, churn
- sample_revenue_large: country, channel, date, revenue, net_revenue

Your task is to generate an SQLite-compatible SQL query to answer the user question.
If time related queries are asked, fetch todays date and then calculate accordingly.
Think step by step and then formulate the query. Do not add any comments in the query.
Generate queries using builtin functions of SQLite.
Maintain consistency in your query generation pattern and generate as simple query as possible for a given question.
If the query cannot be answered from the available datasets, just respond with "Query out of available data range."
Generate the response which contains reasoning step by step and a query in the following format:
{{"Reasoning": ..., "Query": ...}}
User Question: {question}"""
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role":"user","content":prompt}],
)
print(eval(response.choices[0].message.content)["Reasoning"])
print(eval(response.choices[0].message.content)["Query"])

To find the churn for country 'AAA' last month, we need to filter data from the 'sample_sub_details_large' table where the country is 'AAA' and the date is within the last month. First, we'll get the current month's first day using date('now', 'start of month'), then we will obtain the last month's first day using date('now', 'start of month', '-1 month'). This will allow us to filter the records for the entire last month. Finally, we'll sum up the churn values for these filtered records.
SELECT SUM(churn) FROM sample_sub_details_large WHERE country = 'AAA' AND date >= date('now', 'start of month', '-1 month') AND date < date('now', 'start of month')


In [96]:
sql=response.choices[0].message.content.split("```")[1][4:]
print(sql)

SELECT SUM(churn) AS total_churn
FROM sample_sub_details_large
WHERE country = 'AAA' AND date >= date('now', 'start of month', '-1 month') AND date < date('now', 'start of month');



In [99]:
import sqlite3
import pandas as pd
import os

def load_csvs_to_sqlite(db_path):
    conn = sqlite3.connect(db_path)
    # Load each CSV
    bo_df = pd.read_csv('../data/sample_bo_tbl_large.csv')
    bo_df.to_sql('sample_bo_tbl_large', conn, if_exists='replace', index=False)

    sub_df = pd.read_csv('../data/sample_sub_details_large.csv')
    sub_df.to_sql('sample_sub_details_large', conn, if_exists='replace', index=False)

    rev_df = pd.read_csv('../data/sample_revenue_large.csv')
    rev_df.to_sql('sample_revenue_large', conn, if_exists='replace', index=False)

    conn.close()

db_path = os.path.abspath("../db/data.db")
print(db_path)
load_csvs_to_sqlite(db_path)

c:\Users\hp\Documents\lycaassignment\db\data.db


In [82]:
import sqlite3

def run_sql(db_path, sql):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    conn.close()
    print(rows)
    return rows

In [97]:
print(sql)
answer=run_sql(db_path=db_path,sql= sql)

SELECT SUM(churn) AS total_churn
FROM sample_sub_details_large
WHERE country = 'AAA' AND date >= date('now', 'start of month', '-1 month') AND date < date('now', 'start of month');

[(4645,)]


In [98]:
response_prompt =f"""Given the following question and answer, rephrase the answer as a proper meaningful response to the question in a friendly tone.
Question: {question}
Answer: {answer}""" 
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role":"user","content":response_prompt}],
)
print(response.choices[0].message.content)

Last month, the churn in AAA was 4,645.


In [77]:
import sqlite3


conn = sqlite3.connect(db_path)  # adjust path to your db
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:")
for table in tables:
    print(table[0])

conn.close()


Tables in database:
bo_table
sub_table
rev_table
sample_bo_tbl_large
sample_sub_details_large
sample_revenue_large
